In [1]:
# Importing dependencies
import pandas as pd
from sqlalchemy import create_engine
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
# Creating dataframe from csv file
csv_file = "Resources/wine_data.csv"
wine_df=pd.read_csv(csv_file, encoding="utf-8")
wine_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [3]:
#Count of non-null records within each column
wine_df.count()

Unnamed: 0               129971
country                  129908
description              129971
designation               92506
points                   129971
price                    120975
province                 129908
region_1                 108724
region_2                  50511
taster_name              103727
taster_twitter_handle     98758
title                    129971
variety                  129970
winery                   129971
dtype: int64

In [4]:
# Resetting index
wine_df.reset_index()
# Removed unnecessary columns (Description and Designation) and created a copy
new_wine_data = wine_df[["country", "points", "price", "province", "region_1", "region_2", "taster_name",\
                        "taster_twitter_handle", "title", "variety", "winery"]].copy()
new_wine_data.head()

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [5]:
#Checking if there are duplicated records
new_wine_data[new_wine_data.duplicated(keep=False)]

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
9,France,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
10,US,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
11,France,87,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer
12,US,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini
13,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Masseria Setteporte 2012 Rosso (Etna),Nerello Mascalese,Masseria Setteporte
...,...,...,...,...,...,...,...,...,...,...,...
129879,US,90,20.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Maison Bleue 2013 Metis Red (Columbia Valley (...,Red Blend,Maison Bleue
129880,US,90,20.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Martin Ray 2015 Chardonnay (Russian River Valley),Chardonnay,Martin Ray
129881,Spain,90,19.0,Northern Spain,Rueda,NaN,Michael Schachner,@wineschach,Martinsancho 2015 Verdejo (Rueda),Verdejo,Martinsancho
129882,US,90,60.0,Washington,Columbia Valley (WA),Columbia Valley,Sean P. Sullivan,@wawinereport,Matthews 2012 Reserve Red (Columbia Valley (WA)),Bordeaux-style Red Blend,Matthews


In [6]:
# dropping the duplicate records and keeping the first record
new_wine_data=new_wine_data.drop_duplicates(keep='first')
new_wine_data[new_wine_data.duplicated(keep=False)]

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery


In [7]:
# Checking data types for each column
new_wine_data.dtypes

country                   object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [8]:
# Finding the number of unique values in each columns
new_wine_data.nunique()

country                      43
points                       21
price                       390
province                    425
region_1                   1229
region_2                     17
taster_name                  19
taster_twitter_handle        15
title                    118840
variety                     707
winery                    16757
dtype: int64

In [9]:
# Check for NaN in country column
new_wine_data[new_wine_data["country"].isna()==True]

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
913,NaN,87,30.0,NaN,NaN,NaN,Mike DeSimone,@worldwineguys,Gotsa Family Wines 2014 Asureti Valley Chinuri,Chinuri,Gotsa Family Wines
3131,NaN,83,NaN,NaN,NaN,NaN,Roger Voss,@vossroger,Barton & Guestier NV Partager Red,Red Blend,Barton & Guestier
4243,NaN,88,18.0,NaN,NaN,NaN,Mike DeSimone,@worldwineguys,Kakhetia Traditional Winemaking 2012 Red Natur...,Ojaleshi,Kakhetia Traditional Winemaking
9509,NaN,92,28.0,NaN,NaN,NaN,Susan Kostrzewa,@suskostrzewa,Tsililis 2015 Theopetra Malagouzia-Assyrtiko W...,White Blend,Tsililis
9750,NaN,89,28.0,NaN,NaN,NaN,Jeff Jenssen,@worldwineguys,Ross-idi 2015 Orange Nikolaevo Vineyard Chardo...,Chardonnay,Ross-idi
11150,NaN,89,20.0,NaN,NaN,NaN,Jeff Jenssen,@worldwineguys,Orbelus 2013 Melnik,Melnik,Orbelus
11348,NaN,82,NaN,NaN,NaN,NaN,Roger Voss,@vossroger,Barton & Guestier NV Partager White,White Blend,Barton & Guestier
14030,NaN,88,25.0,NaN,NaN,NaN,Jeff Jenssen,@worldwineguys,St. Donat 2013 Márga White,White Blend,St. Donat
16000,NaN,86,40.0,NaN,NaN,NaN,Michael Schachner,@wineschach,Familia Deicas 2015 Valle de los Manantiales V...,Tannat,Familia Deicas
16749,NaN,91,NaN,NaN,NaN,NaN,Lauren Buzzeo,@laurbuzz,Bartho Eksteen 2016 Cape Winemakers Guild Vloe...,Sauvignon Blanc,Bartho Eksteen


In [10]:
# Dropped null records in country column
new_wine_data=new_wine_data.loc[(new_wine_data["country"].isna()==False),:]
new_wine_data.head()

,country,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [11]:
#Checking if null values in country column has been dropped
new_wine_data["country"].isna().sum()

0

In [1]:
#Creating a year column from title column
new_wine_data["year"]=new_wine_data["title"].str.extract('(\s[0-9]{4}\s)')
new_wine_data["year"]=new_wine_data["year"].str.strip()
new_wine_data.head(50)

NameError: name 'new_wine_data' is not defined

In [13]:
new_wine_data.count()

country                  119841
points                   119841
price                    111453
province                 119841
region_1                 100344
region_2                  46754
taster_name               94942
taster_twitter_handle     90415
title                    119841
variety                  119840
winery                   119841
year                     115651
dtype: int64

In [14]:
#Creating a location column by combining country, province, region_1, region_2
new_wine_data["location"]=new_wine_data["region_1"].fillna('-').map(str)\
+ ", "+ new_wine_data["region_2"].fillna('-')\
+ ", "+ new_wine_data["province"].fillna('-')\
+ ", "+ new_wine_data["country"]

new_wine_data.count()

country                  119841
points                   119841
price                    111453
province                 119841
region_1                 100344
region_2                  46754
taster_name               94942
taster_twitter_handle     90415
title                    119841
variety                  119840
winery                   119841
year                     115651
location                 119841
dtype: int64

In [15]:
#URL to fetch secondary data- grape variety by color
url="https://en.wikipedia.org/wiki/List_of_grape_varieties#Red_table_grapes"

In [16]:
#Reading the data from html into a list of tables
tables=pd.read_html(url)

In [17]:
#Creating data frames for each grape color
#Dataframe for red grapes
grapes_red_df=tables[0]
#Dataframe for white grapes
grapes_white_df=tables[1]
#Dataframe for rose' grapes
grapes_rose_df=tables[2]

In [18]:
#Assigning the grape_color column for red grapes
grapes_red_df["grape_color"]="red"

#Assigning the grape_color column for white grapes
grapes_white_df["grape_color"]="white"

#Assigning the grape_color column for rose
grapes_rose_df["grape_color"]="rose"

In [19]:
#Combining the three different grape color dataframes into one 
grape_color_df=pd.concat([grapes_red_df,grapes_white_df,grapes_rose_df])
grape_color_df=grape_color_df[["Common Name(s)","grape_color"]]
grape_color_df["grape_color"].value_counts()

red      426
white    394
rose       2
Name: grape_color, dtype: int64

In [20]:
#Dropping duplicated records
grape_color_df=grape_color_df.drop_duplicates()
#Finding if we have multiple records with same grape but different color
grape_color_df[grape_color_df.duplicated(subset=["Common Name(s)"],keep=False)]
#grape_color_df[grape_color_df["Common Name(s)"]=='Alexandroouli']

,Common Name(s),grape_color
34,Arnsburger,red
156,Cereza,red
222,Grignolino,red
332,Pignerol,red
368,Schönburger,red
27,Arnsburger,white
87,Cereza,white
170,Grignolino,white
275,Pignerol,white
320,Schönburger,white


In [21]:
#Dropping duplicate records based on Common Name(s) and keeping only the first record
grape_color_df=grape_color_df.drop_duplicates(subset='Common Name(s)', keep="first")
grape_color_df[grape_color_df.duplicated(subset=["Common Name(s)"],keep=False)]

,Common Name(s),grape_color


In [22]:
# Merging wine dataframe with grape color dataframe
wine_review_df=new_wine_data.merge(grape_color_df, left_on="variety",right_on="Common Name(s)",how="left")

In [23]:
#Creating a wine review dataframe with the grape color column added for red color grapes
wine_review_df=wine_review_df[["country", "points", "price", "province", "region_1", "region_2","location", "taster_name",\
                        "taster_twitter_handle", "title","year","variety", "winery","grape_color"]]

In [24]:
#Checking for null values in the grape_color column
wine_review_df[wine_review_df["grape_color"].isna()==True]

,country,points,price,province,region_1,region_2,location,taster_name,taster_twitter_handle,title,year,variety,winery,grape_color
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,"Etna, -, Sicily & Sardinia, Italy",Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),2013,White Blend,Nicosia,NaN
1,Portugal,87,15.0,Douro,NaN,NaN,"-, -, Douro, Portugal",Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),2011,Portuguese Red,Quinta dos Avidagos,NaN
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,"Willamette Valley, Willamette Valley, Oregon, US",Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),2013,Pinot Gris,Rainstorm,NaN
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,"Lake Michigan Shore, -, Michigan, US",Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,2013,Riesling,St. Julian,NaN
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,"Willamette Valley, Willamette Valley, Oregon, US",Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,2012,Pinot Noir,Sweet Cheeks,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119836,Germany,90,28.0,Mosel,NaN,NaN,"-, -, Mosel, Germany",Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,2013,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),NaN
119837,US,90,75.0,Oregon,Oregon,Oregon Other,"Oregon, Oregon Other, Oregon, US",Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),2004,Pinot Noir,Citation,NaN
119838,France,90,30.0,Alsace,Alsace,NaN,"Alsace, -, Alsace, France",Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,2013,Gewürztraminer,Domaine Gresser,NaN
119839,France,90,32.0,Alsace,Alsace,NaN,"Alsace, -, Alsace, France",Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),2012,Pinot Gris,Domaine Marcel Deiss,NaN


In [25]:
#Checking the count of values for each grape color
wine_review_df["grape_color"].value_counts()

red      21201
white    14774
Name: grape_color, dtype: int64

In [26]:
#Rename columns in the data set to match column names in the table
wine_review_df.columns=["country_name","wine_score","wine_price","province","region1","region2",\
                        "combined_region","taster_name","taster_twitter","wine_name","wine_year","grape_name",\
                        "winery_name", "grape_color"]

In [27]:
wine_review_df.head()

,country_name,wine_score,wine_price,province,region1,region2,combined_region,taster_name,taster_twitter,wine_name,wine_year,grape_name,winery_name,grape_color
0,Italy,87,NaN,Sicily & Sardinia,Etna,NaN,"Etna, -, Sicily & Sardinia, Italy",Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),2013,White Blend,Nicosia,NaN
1,Portugal,87,15.0,Douro,NaN,NaN,"-, -, Douro, Portugal",Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),2011,Portuguese Red,Quinta dos Avidagos,NaN
2,US,87,14.0,Oregon,Willamette Valley,Willamette Valley,"Willamette Valley, Willamette Valley, Oregon, US",Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),2013,Pinot Gris,Rainstorm,NaN
3,US,87,13.0,Michigan,Lake Michigan Shore,NaN,"Lake Michigan Shore, -, Michigan, US",Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,2013,Riesling,St. Julian,NaN
4,US,87,65.0,Oregon,Willamette Valley,Willamette Valley,"Willamette Valley, Willamette Valley, Oregon, US",Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,2012,Pinot Noir,Sweet Cheeks,NaN


In [28]:
#Checking integrity of new wine review data set
wine_review_df.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
dtype: int64

In [29]:
#Check for duplicated records
wine_review_df[wine_review_df.duplicated()]

,country_name,wine_score,wine_price,province,region1,region2,combined_region,taster_name,taster_twitter,wine_name,wine_year,grape_name,winery_name,grape_color


In [30]:
wine_review_df.nunique()

country_name           43
wine_score             21
wine_price            390
province              425
region1              1229
region2                17
combined_region      1619
taster_name            19
taster_twitter         15
wine_name          118781
wine_year              82
grape_name            701
winery_name         16745
grape_color             2
dtype: int64

In [31]:
#Finding max length of data in each column in the dataframe so that the column length in tables can be gauged

dict([(v, wine_review_df[v].apply(lambda r: len(str(r)) if r!=None else 0).max())for v in wine_review_df.columns.values])

{'country_name': 22,
 'wine_score': 3,
 'wine_price': 6,
 'province': 31,
 'region1': 50,
 'region2': 17,
 'combined_region': 84,
 'taster_name': 18,
 'taster_twitter': 16,
 'wine_name': 136,
 'wine_year': 4,
 'grape_name': 35,
 'winery_name': 54,
 'grape_color': 5}

### Creating master tables for loading


In [32]:
#Creating country_df that will be used to load country table into the database
country_df=pd.DataFrame(wine_review_df["country_name"].unique())

#Set index to start at 1
country_df.index = country_df.index + 1

In [33]:
#Resetting the index to create primary key for table
country_df=country_df.reset_index()

In [34]:
#Renaming columns to match column names in country table
country_df.columns=["country_id","country_name"]
country_df.nunique()

country_id      43
country_name    43
dtype: int64

In [35]:
#Creating taster_df that will be used to load taster table into the database
taster_df=wine_review_df[["taster_name", "taster_twitter"]].drop_duplicates().reset_index().drop(columns=["index"])

#Set index to start at 1
taster_df.index = taster_df.index + 1

In [36]:
#Resetting the index to create primary key for table
taster_df=taster_df.reset_index()

In [37]:
#Renaming columns to match column names in taster table
taster_df.columns=["taster_id","taster_name","taster_twitter"]
taster_df.nunique()

taster_id         20
taster_name       19
taster_twitter    15
dtype: int64

In [38]:
taster_df

,taster_id,taster_name,taster_twitter
0,1,Kerin O’Keefe,@kerinokeefe
1,2,Roger Voss,@vossroger
2,3,Paul Gregutt,@paulgwine
3,4,Alexander Peartree,NaN
4,5,Michael Schachner,@wineschach
5,6,Anna Lee C. Iijima,NaN
6,7,Virginie Boone,@vboone
7,8,Matt Kettmann,@mattkettmann
8,9,NaN,NaN
9,10,Sean P. Sullivan,@wawinereport


In [39]:
#Creating winery_df that will be used to load winery table into the database
winery_df=wine_review_df[["winery_name"]].drop_duplicates().reset_index().drop(columns=["index"])

#Set index to start at 1
winery_df.index = winery_df.index + 1

In [40]:
#Resetting the index to create primary key for table
winery_df=winery_df.reset_index()

In [41]:
#Renaming columns to match column names in winery table
winery_df.columns=["winery_id","winery_name"]
winery_df.nunique()

winery_id      16745
winery_name    16745
dtype: int64

In [42]:
#Creating grape_variety_df that will be used to load grape_variety table into the database
grape_variety_df=wine_review_df[["grape_name","grape_color"]].drop_duplicates().reset_index().drop(columns=["index"])

#Set index to start at 1
grape_variety_df.index = grape_variety_df.index + 1

In [43]:
#Resetting the index to create primary key for table
grape_variety_df=grape_variety_df.reset_index()


In [44]:
#Renaming columns to match column names in winery table
grape_variety_df.columns=["grape_id","grape_name","grape_color"]
grape_variety_df.nunique()

grape_id       702
grape_name     701
grape_color      2
dtype: int64

In [45]:
#Creating region_df that will be used to load region table into the database
region_df=wine_review_df[["country_name","province","region1","region2","combined_region"]].drop_duplicates()\
.reset_index().drop(columns=["index"])

#Set index to start at 1
region_df.index = region_df.index + 1

In [46]:
#Resetting the index to create primary key for table
region_df=region_df.reset_index()

In [47]:
#Renaming columns to match column names in region table
region_df.columns=["region_id","country_name","province","region1","region2","combined_region"]
region_df.nunique()

region_id          1619
country_name         43
province            425
region1            1229
region2              17
combined_region    1619
dtype: int64

In [48]:
#Creating a province dataframe of unique provinces and will be used to find latitude and longitudes for the provinces
province_df=pd.DataFrame(region_df["province"].unique())
province_df.columns=["province"]
province_df.head()

,province
0,Sicily & Sardinia
1,Douro
2,Oregon
3,Michigan
4,Northern Spain


In [49]:
# Calculating latitude and longtitude for provinces in province dataframe and assigning to latitude and longitude list

longitude = [] 
latitude = [] 
   
# function to find the coordinate 
# of a given city  
def findGeocode(province): 
       
    # try and catch is used to overcome 
    # the exception thrown by geolocator 
    # using geocodertimedout   
    #try: 
          
    # Specify the user_agent as your 
    # app name it should not be none 
    geolocator = Nominatim(user_agent="wine_review") 

    return geolocator.geocode(province,timeout=None) 
      
    #except GeocoderTimedOut: 
          
        #return findGeocode(province)     
  
    # each value from city column 
    # will be fetched and sent to 
    # function find_geocode    
for i in (province_df["province"]): 
      
    if findGeocode(i) != None: 
           
        loc = findGeocode(i) 
          
        # coordinates returned from  
        # function is stored into 
        # two separate list 
        latitude.append(loc.latitude) 
        longitude.append(loc.longitude) 
       
    # if coordinate for a city not 
    # found, insert "NaN" indicating  
    # missing value  
    else: 
        latitude.append(np.nan) 
        longitude.append(np.nan) 

In [50]:
#Assigning latitudes and longitudes created for province to respective columns in province dataframe
province_df["latitude"]=latitude
province_df["longitude"]=latitude

In [51]:
#Merging region and province dataframe to add latitude and longitude columns to region dataframe
region_df=region_df.merge(region_df.merge(province_df, how='inner', on='province', sort=False))
region_df.nunique()

region_id          1619
country_name         43
province            425
region1            1229
region2              17
combined_region    1619
latitude            372
longitude           372
dtype: int64

In [52]:
#Merging region and country dataframe to add country_id column to region dataframe and dropping country_name column
region_df=region_df.merge(region_df.merge(country_df, how='inner', on='country_name', sort=False))\
.drop(columns=["country_name"])


In [53]:
region_df.nunique()

region_id          1619
province            425
region1            1229
region2              17
combined_region    1619
latitude            372
longitude           372
country_id           43
dtype: int64

In [54]:
wine_review_df.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
dtype: int64

In [55]:
#Creating wine_review_data_df dataframe from wine_review_df dataframe and adding references to other dataframe

#Adding grape_id column in wine_review_data_df which is referenced to grape dataframe
wine_review_data_df=wine_review_df.merge(grape_variety_df,on=["grape_name","grape_color"],sort=False)
wine_review_data_df.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
grape_id           119841
dtype: int64

In [56]:
#Adding region_id column in wine_review_data_df which is referenced to region dataframe
wine_review_data_df=wine_review_data_df.merge(region_df[["region_id","combined_region"]]\
                                                              ,on="combined_region",sort=False)
wine_review_data_df.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
grape_id           119841
region_id          119841
dtype: int64

In [57]:
#Adding taster_id column in wine_review_data_df which is referenced to taster dataframe
wine_review_data_df=wine_review_data_df.merge(taster_df,on=["taster_name","taster_twitter"],sort=False)
wine_review_data_df.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
grape_id           119841
region_id          119841
taster_id          119841
dtype: int64

In [58]:
#Adding winery_id column in wine_review_data_df which is referenced to winery dataframe
final_wine_data=pd.merge(wine_review_data_df, winery_df, on="winery_name", how="inner", sort=False)
final_wine_data.count()

country_name       119841
wine_score         119841
wine_price         111453
province           119841
region1            100344
region2             46754
combined_region    119841
taster_name         94942
taster_twitter      90415
wine_name          119841
wine_year          115651
grape_name         119840
winery_name        119841
grape_color         35975
grape_id           119841
region_id          119841
taster_id          119841
winery_id          119841
dtype: int64

In [59]:
final_wine_review_df=final_wine_data[["wine_name","wine_year","wine_price","wine_score",\
                                 "grape_id","region_id","taster_id","winery_id"]].drop_duplicates()\
.reset_index().drop(columns=["index"])

#Set index to start at 1
final_wine_review_df.index = final_wine_review_df.index + 1

In [60]:
final_wine_review_df=final_wine_review_df.reset_index()

In [61]:
final_wine_review_df.columns=["wine_review_id","wine_name","wine_year","wine_price","wine_score",\
                                 "grape_id","region_id","taster_id","winery_id"]

In [62]:
final_wine_review_df.dtypes

wine_review_id      int64
wine_name          object
wine_year          object
wine_price        float64
wine_score          int64
grape_id            int64
region_id           int64
taster_id           int64
winery_id           int64
dtype: object

In [63]:
final_wine_review_df.head()

,wine_review_id,wine_name,wine_year,wine_price,wine_score,grape_id,region_id,taster_id,winery_id
0,1,Nicosia 2013 Vulkà Bianco (Etna),2013,NaN,87,1,1,1,1
1,2,Nicosia 2013 Fondo Filara Bianco (Etna),2013,NaN,88,1,1,1,1
2,3,Nicosia 2011 Rosso (Etna),2011,NaN,87,15,1,1,1
3,4,Nicosia 2012 Vulkà Rosso (Etna),2012,NaN,91,15,1,1,1
4,5,Nicosia 2011 Fondo Filara (Cerasuolo di Vitto...,2011,15.0,90,15,522,1,1


## Connect to database

In [64]:
rds_connection_string = "postgres:postgres@localhost:5433/wine_review_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

## Check for Tables

In [65]:
engine.table_names()

['grape_variety', 'region', 'taster', 'winery', 'country', 'wine_review']

## Inserting data into tables

In [66]:
#Inserting data into country table
country_df.to_sql(name='country', con=engine, if_exists='append', index=False)

In [67]:
#Inserting data into taster table
taster_df.to_sql(name='taster', con=engine, if_exists='append', index=False)

In [68]:
#Inserting data into winery table
winery_df.to_sql(name='winery', con=engine, if_exists='append', index=False)

In [69]:
#Inserting data into grape_variety table
grape_variety_df.to_sql(name='grape_variety', con=engine, if_exists='append', index=False)

In [70]:
#Inserting data into region table
region_df.to_sql(name='region', con=engine, if_exists='append', index=False)

In [71]:
#Inserting data into wine_review table
final_wine_review_df.to_sql(name='wine_review', con=engine, if_exists='append', index=False)